In [2]:
import json
import re, os, argparse, pdfplumber
from pathlib import Path
from tqdm import tqdm
import logging
from PyPDF2 import PdfReader, PdfWriter

for name in ("pdfminer", "pdfminer.layout", "pdfminer.pdfpage"):
    logging.getLogger(name).setLevel(logging.ERROR)

In [3]:
PATH_GLOBAL = os.getcwd()
PATH = os.path.join(PATH_GLOBAL, "datasets")


# Prueba inicial con solo un mes (febrero 2024 - 10 fallos)
PATH_FALLOS = Path(os.path.join(PATH, "2024"))
PATH_RECORTES = Path(os.path.join(PATH, "cropped_pdfs"))
PATH_TXT = Path(os.path.join(PATH, "fallos_txts"))
PATH_JSON = Path(os.path.join(PATH, "fallos_json"))

os.makedirs(PATH_RECORTES, exist_ok=True)
os.makedirs(PATH_TXT, exist_ok=True)
os.makedirs(PATH_JSON, exist_ok=True)

## Recorte de PDFs

In [4]:
HEADER_PT = 82      # quitar SOLO en páginas impares (arriba)
FOOTER_PT = 50      # quitar SIEMPRE (abajo)

for pdf in PATH_FALLOS.glob("*.pdf"):
    reader, writer = PdfReader(str(pdf)), PdfWriter()
    for idx, page in enumerate(reader.pages, start=1):
        box = page.cropbox                       # caja de recorte base
        # -- recorte inferior --
        box.lower_left  = (box.lower_left[0],  box.lower_left[1] + FOOTER_PT)
        # -- recorte superior en páginas impares --
        if idx % 2 == 1:
            box.upper_right = (box.upper_right[0], box.upper_right[1] - HEADER_PT)

        # aplicar a todos los bounding-boxes que respetan los visores/lectores
        page.cropbox  = box
        page.trimbox  = box
        page.mediabox = box
        writer.add_page(page)

    out_file = PATH_RECORTES / pdf.name
    with out_file.open("wb") as f:
        writer.write(f)

    print("✔ Cropped →", out_file)

In [5]:
HEADER_PT = 82      # quitar SOLO en páginas impares (arriba)
FOOTER_PT = 50      # quitar SIEMPRE (abajo)

for pdf in PATH_FALLOS.rglob("*.pdf"):  # Cambio aquí: .rglob() en lugar de .glob()
    reader, writer = PdfReader(str(pdf)), PdfWriter()
    for idx, page in enumerate(reader.pages, start=1):
        box = page.cropbox                       # caja de recorte base
        # -- recorte inferior --
        box.lower_left  = (box.lower_left[0],  box.lower_left[1] + FOOTER_PT)
        # -- recorte superior en páginas impares --
        if idx % 2 == 1:
            box.upper_right = (box.upper_right[0], box.upper_right[1] - HEADER_PT)

        # aplicar a todos los bounding-boxes que respetan los visores/lectores
        page.cropbox  = box
        page.trimbox  = box
        page.mediabox = box
        writer.add_page(page)

    # Mantener la estructura de subcarpetas en el output
    rel_path = pdf.relative_to(PATH_FALLOS)
    out_file = PATH_RECORTES / rel_path
    out_file.parent.mkdir(parents=True, exist_ok=True)  # Crear subcarpetas si no existen
    
    with out_file.open("wb") as f:
        writer.write(f)

    print("✔ Cropped →", out_file)

✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8996.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8613.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8997.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8940.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8947.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8953.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8984.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8985.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/03/8946.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/T

PdfReadError("Invalid Elementary Object starting with b'h' @53984: b'pe/Action/S/URI/URI https://consultarjurisprudencia.justicialapampa.gob.ar/juris'")
PdfReadError("Invalid Elementary Object starting with b'p' @54044: b'lapampa.gob.ar/jurisprudencia/Home/Detalle/34167 >>\\r\\n>> \\rendobj\\r\\n10 0 obj\\r<<\\r\\n/T'")
PdfReadError("Invalid Elementary Object starting with b'h' @54198: b'pe/Action/S/URI/URI https://consultarjurisprudencia.justicialapampa.gob.ar/juris'")
PdfReadError("Invalid Elementary Object starting with b'p' @54258: b'lapampa.gob.ar/jurisprudencia/Home/Detalle/34167 >>\\r\\n>> \\rendobj\\r\\n3 0 obj\\r<<\\r\\n/Ty'")


✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9096.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/8983.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/8206.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9084.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/8935.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9155.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9141.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9194.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/TP_NLP/datasets/cropped_pdfs/12/9180.pdf
✔ Cropped → /Users/gaborgorondi/Documents/udesa/8_nlp/tp_final/T

# NORMALIZATION

In [6]:
PATTERNS = [
    r'^Superior Tribunal.*$',
    r'^Sala Civil y Comercial.*$',
    r'^\s*\d+\s*$',
    r'^Poder Judicial.*$',
    r'^Firmado digitalmente.*$',
    r'^Página \s*\d+(\s*de\s*\d+)?',
    r'^\s*\d{4}-\d{2}-\d{2}T\d{2}:',
]
REGEX = re.compile('|'.join(PATTERNS), re.IGNORECASE)


def join_lines_to_paragraphs(text):
    lines = text.split('\n')
    paragraphs = []
    current_para = ""
    for line in lines:
        line = line.strip()
        if not line:
            # Línea vacía indica fin de párrafo
            if current_para:
                paragraphs.append(current_para.strip())
                current_para = ""
        else:
            # Si la línea anterior no termina con punto pero la linea actual no es solamente mayuscula y termina con ':', unir
            if current_para and not current_para.endswith(('.', ':', '?', '!', ';')):
                if line.isupper() and line.endswith(':') and len(line) < 50:
                    # Si la línea es mayúscula y termina con ':', no unir
                    paragraphs.append(current_para.strip())
                    current_para = line
                else:
                    current_para += " " + line
            else:
                if current_para:
                    paragraphs.append(current_para.strip())
                current_para = line
    if current_para:
        paragraphs.append(current_para.strip())
    return "\n\n".join(paragraphs)

def clean_page(text: str) -> str:
    return '\n'.join([ln for ln in (text or "").splitlines() if not REGEX.match(ln)]).strip()


## Normalization to .TXT:

In [7]:
def clean_pdf(pdf_path: Path) -> str:
    pages_clean = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for page in pdf.pages:
            page_text = clean_page(page.extract_text())
            page_text = join_lines_to_paragraphs(page_text)  # <--- aplicar aquí
            pages_clean.append(page_text)
    return "\n\n".join(pages_clean)


def main(pdf_dir: str, out_dir: str):
    pdf_root = Path(pdf_dir).resolve()
    out_root = Path(out_dir).resolve()
    out_root.mkdir(parents=True, exist_ok=True)

    pdf_files = list(pdf_root.rglob("*.pdf"))
    if not pdf_files:
        print(f"No se encontraron PDFs en {pdf_dir}")
        return

    for path in tqdm(pdf_files, desc="Cleaning PDFs"):
        # —— NUEVO: ruta relativa para reproducir subcarpetas ——
        rel_path = path.relative_to(pdf_root).with_suffix(".txt")
        dst = out_root / rel_path
        dst.parent.mkdir(parents=True, exist_ok=True)  # crea la subcarpeta si falta
        # --------------------------------------------------------
        dst.write_text(clean_pdf(path), encoding="utf-8")

main(PATH_RECORTES, PATH_TXT)

Cleaning PDFs: 100%|██████████| 297/297 [01:11<00:00,  4.15it/s]


## Normalization to JSON:

In [8]:
def extract_materia_preliminar(inicio_paragraphs):
    """
    Busca dentro de los párrafos de INICIO el texto entre comillas,
    y extrae el texto que sigue justo después de 'S/' dentro de esa cadena.
    """
    import re

    for para in inicio_paragraphs:
        # Buscamos todas las cadenas entre comillas
        quoted_texts = re.findall(r'"([^"]+)"', para)
        for qt in quoted_texts:
            # Buscamos 'S/' y capturamos lo que viene después hasta el fin o hasta un guion, coma o fin de línea
            m = re.search(r'S/\s*([^\-\,]+)', qt, re.IGNORECASE)
            if m:
                materia = m.group(1).strip()
                return materia.upper()
    return None

def is_key_line(line: str) -> bool:
    # Es key si toda mayúscula, termina con ':' y longitud < 50 (para evitar líneas muy largas)
    return line.isupper() and line.endswith(':') and len(line) < 50

def split_into_sections(text: str) -> dict:
    """
    Convierte el texto en dict con keys y listas de párrafos.
    La key 'INICIO' contiene todo lo previo a la primera key.
    """
    lines = text.split('\n\n')  # separar párrafos
    sections = {}
    current_key = 'INICIO'
    sections[current_key] = []

    for para in lines:
        para_strip = para.strip()
        if is_key_line(para_strip):
            current_key = para_strip[:-1]  # sacamos ':'
            if current_key not in sections:
                sections[current_key] = []
        else:
            sections[current_key].append(para_strip)
    return sections



def clean_pdf_no_join(pdf_path: Path) -> str:
    """
    Extrae el texto limpio de todo el PDF, concatenando páginas
    sin unir líneas en párrafos todavía para evitar cortar párrafos.
    """
    pages_text = []
    with pdfplumber.open(str(pdf_path)) as pdf:
        for page in pdf.pages:
            page_text = clean_page(page.extract_text())
            pages_text.append(page_text.strip())
    # Unir páginas con salto de línea simple para no cortar párrafos
    return "\n".join(pages_text)

def clean_pdf_to_sections_structured(pdf_path: Path) -> dict:
    full_text = clean_pdf_no_join(pdf_path)
    full_text = join_lines_to_paragraphs(full_text)
    sections = split_into_sections(full_text)
    materia = extract_materia_preliminar(sections.get('INICIO', []))
    
    return {
        "INFORMACION": {
            "MATERIA_PRELIMINAR": materia or "",
            "RESUMEN": ""
        },
        "CONTENIDO": sections
    }


def main(pdf_dir: str, out_dir: str):
    pdf_root = Path(pdf_dir).resolve()
    out_root = Path(out_dir).resolve()
    out_root.mkdir(parents=True, exist_ok=True)

    pdf_files = list(pdf_root.rglob("*.pdf"))
    if not pdf_files:
        print(f"No se encontraron PDFs en {pdf_dir}")
        return

    for path in tqdm(pdf_files, desc="PDF to JSON:"):
        rel_path = path.relative_to(pdf_root).with_suffix(".json")
        dst = out_root / rel_path
        dst.parent.mkdir(parents=True, exist_ok=True)
        
        sections_structured = clean_pdf_to_sections_structured(path)
        dst.write_text(json.dumps([sections_structured], ensure_ascii=False, indent=2), encoding="utf-8")


main(PATH_RECORTES, PATH_JSON)

PDF to JSON:: 100%|██████████| 297/297 [01:11<00:00,  4.17it/s]


## JSON Analysis

In [9]:
def analyze_json_structures(json_dir: str):
    """
    Analiza la estructura de todos los JSONs extraídos:
    - Cuenta total de archivos
    - Lista todas las keys encontradas
    - Cuenta frecuencia de cada key
    - Muestra estadísticas de estructura
    """
    import json
    from collections import Counter, defaultdict
    from pathlib import Path
    
    json_root = Path(json_dir).resolve()
    json_files = list(json_root.rglob("*.json"))
    
    if not json_files:
        print(f"No se encontraron JSONs en {json_dir}")
        return
    
    print(f"📊 ANÁLISIS DE ESTRUCTURA - {len(json_files)} archivos JSON")
    print("=" * 60)
    
    # Contadores para análisis
    contenido_keys = Counter()
    informacion_keys = Counter()
    total_sections_per_file = []
    files_with_materia = 0
    
    for json_path in json_files:
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)[0]  # Primer elemento de la lista
            
            # Analizar INFORMACION
            if 'INFORMACION' in data:
                for key in data['INFORMACION'].keys():
                    informacion_keys[key] += 1
                
                # Contar archivos con materia
                materia = data['INFORMACION'].get('MATERIA_PRELIMINAR', '')
                if materia and materia.strip():
                    files_with_materia += 1
            
            # Analizar CONTENIDO
            if 'CONTENIDO' in data:
                sections = data['CONTENIDO']
                total_sections_per_file.append(len(sections))
                
                for key in sections.keys():
                    contenido_keys[key] += 1
                    
        except Exception as e:
            print(f"❌ Error procesando {json_path}: {e}")
    
    # RESULTADOS
    print(f"🗂️  ARCHIVOS PROCESADOS: {len(json_files)}")
    print(f"📋 ARCHIVOS CON MATERIA: {files_with_materia}")
    print()
    
    print("🔧 KEYS EN 'INFORMACION':")
    for key, count in informacion_keys.most_common():
        percentage = (count / len(json_files)) * 100
        print(f"  • {key}: {count} archivos ({percentage:.1f}%)")
    print()
    
    print("📖 SECCIONES EN 'CONTENIDO' (Top 20):")
    for key, count in contenido_keys.most_common(20):
        percentage = (count / len(json_files)) * 100
        print(f"  • {key}: {count} archivos ({percentage:.1f}%)")
    
    if len(contenido_keys) > 20:
        print(f"  ... y {len(contenido_keys) - 20} secciones más")
    print()
    
    # Estadísticas de secciones por archivo
    if total_sections_per_file:
        avg_sections = sum(total_sections_per_file) / len(total_sections_per_file)
        min_sections = min(total_sections_per_file)
        max_sections = max(total_sections_per_file)
        
        print("📊 ESTADÍSTICAS DE SECCIONES POR ARCHIVO:")
        print(f"  • Promedio: {avg_sections:.1f} secciones")
        print(f"  • Mínimo: {min_sections} secciones")
        print(f"  • Máximo: {max_sections} secciones")
        print()
    
    print(f"📋 TOTAL DE SECCIONES ÚNICAS ENCONTRADAS: {len(contenido_keys)}")
    
    return {
        'total_files': len(json_files),
        'files_with_materia': files_with_materia,
        'contenido_keys': dict(contenido_keys),
        'informacion_keys': dict(informacion_keys),
        'sections_stats': {
            'avg': avg_sections if total_sections_per_file else 0,
            'min': min_sections if total_sections_per_file else 0,
            'max': max_sections if total_sections_per_file else 0
        }
    }

# Ejecutar análisis
analysis_results = analyze_json_structures(PATH_JSON)

📊 ANÁLISIS DE ESTRUCTURA - 297 archivos JSON
🗂️  ARCHIVOS PROCESADOS: 297
📋 ARCHIVOS CON MATERIA: 273

🔧 KEYS EN 'INFORMACION':
  • MATERIA_PRELIMINAR: 297 archivos (100.0%)
  • RESUMEN: 297 archivos (100.0%)

📖 SECCIONES EN 'CONTENIDO' (Top 20):
  • INICIO: 297 archivos (100.0%)
  • RESUELVE: 294 archivos (99.0%)
  • Y VISTO: 145 archivos (48.8%)
  • ACUERDO: 134 archivos (45.1%)
  • CONSIDERANDO: 85 archivos (28.6%)
  • VISTO: 84 archivos (28.3%)
  • VISTO Y CONSIDERANDO: 66 archivos (22.2%)
  • FEDERICO TEPSICH DIJO: 42 archivos (14.1%)
  • VOCAL GISELA N. SCHUMACHER DIJO: 37 archivos (12.5%)
  • GISELA N. SCHUMACHER DIJO: 34 archivos (11.4%)
  • CARLOS FEDERICO TEPSICH DIJO: 34 archivos (11.4%)
  • HONORARIOS PROFESIONALES: 28 archivos (9.4%)
  • LEONARDO PORTELA DIJO: 26 archivos (8.8%)
  • N. SCHUMACHER DIJO: 16 archivos (5.4%)
  • SCHUMACHER DIJO: 14 archivos (4.7%)
  • DIJERON: 14 archivos (4.7%)
  • VOCAL GISELA SCHUMACHER DIJO: 14 archivos (4.7%)
  • SCHUMACHER DIJERON: 13 ar